### 1- Comparacion entre secuencial y en paralelo

In [1]:
from dask.distributed import Client

client = Client("scheduler:8786")

In [2]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
from dask import delayed
from dask import dataframe

Creacion de base de datos en pandas para secuencial

In [3]:
# Base en pandas
trips_pd = pd.read_csv("/data/trips.csv")
trips_pd['y'] = trips_pd['tip_amount'] / trips_pd['fare_amount']
trips_pd = trips_pd[np.isfinite(trips_pd['y'])]
trips_pd = pd.get_dummies(trips_pd, columns=["car_type"])
trips_pd = trips_pd.drop(columns=['taxi_id', 'tip_amount', 'tpep_dropoff_datetime','tpep_pickup_datetime'])

X_seq = trips_pd.drop(columns=['y'])
y_seq = trips_pd['y']

trips_pd.head()

,fare_amount,passenger_count,trip_distance,y,car_type_A,car_type_B
0,22.0,1,6.90,0.209091,1,0
1,9.0,1,1.81,0.000000,1,0
2,7.5,1,0.96,0.133333,1,0
3,8.5,1,1.90,0.117647,1,0
4,7.5,1,1.00,0.221333,1,0


Creacion de base de datos en dask para paralelo

In [4]:
# Base en dask
trips_df = dataframe.read_csv("/data/trips.csv")

trips_df = trips_df.assign(y = trips_df.tip_amount/trips_df.fare_amount)
trips_df = trips_df.assign(car_type_A = trips_df.car_type == 'A')
trips_df = trips_df.assign(car_type_B = trips_df.car_type == 'B')
trips_df.car_type_A = trips_df.car_type_A.astype('int')
trips_df.car_type_B = trips_df.car_type_B.astype('int')
trips_df = trips_df.drop(['car_type','taxi_id','tip_amount','tpep_dropoff_datetime','tpep_pickup_datetime'],axis=1)
trips_df = trips_df.dropna()

X_dask = trips_df.drop('y',axis=1)
y_dask = trips_df.y

trips_df.head()

# Se guarda en CSV para la parte de Spark
dataframe.to_csv(trips_df,"/data/")

['/data/0.part']

Creacion de grids

In [5]:
%%time 
grid_params = {
    "alpha": [0.01, 0.03, 0.05, 0.07, 0.09, 0.1]}
ridge = Ridge(max_iter=100)
grid = GridSearchCV(estimator=ridge, 
                    param_grid=grid_params, 
                    n_jobs=-1, cv = 10, scoring="neg_mean_squared_error")

delayed_grid = delayed(grid)

CPU times: user 206 µs, sys: 54 µs, total: 260 µs
Wall time: 246 µs


Ejecucion en secuencial

In [13]:
%%time
grid_seq = grid.fit(X_seq, y_seq)

CPU times: user 114 ms, sys: 34.5 ms, total: 149 ms
Wall time: 590 ms


Ejecucion en paralelo

In [10]:
%%time
grid_paralelo = delayed_grid.fit(X_dask,y_dask).compute()

CPU times: user 9.51 ms, sys: 4.67 ms, total: 14.2 ms
Wall time: 488 ms


Se compara la perdida y mejor modelo para ver que el resultado fue el mismo

In [14]:
print("Loss: %f Mejor modelo: %s"%(grid_seq.best_score_, grid_seq.best_params_))
print("Loss: %f Mejor modelo: %s"%(grid_paralelo.best_score_, grid_paralelo.best_params_))

Loss: -0.016417 Mejor modelo: {'alpha': 0.1}
Loss: -0.016417 Mejor modelo: {'alpha': 0.1}


### 2- Dask ML

In [15]:
from dask_ml.linear_model import LinearRegression as LinearRegression_dask
from sklearn.externals.joblib import parallel_backend
from dask_searchcv import GridSearchCV as GridSearchCV_dask 
import dask.array as da

X_ml = X_dask.values
y_ml = y_dask.values    

Primero se implementa utilizando la funcion LinearRegression de dask

In [16]:
%%time
param_grid = {
    "C": [0.01, 0.03, 0.05, 0.07, 0.09, 0.1]
}
ridge = LinearRegression_dask(max_iter= 100, penalty='l2')
grid_ml = GridSearchCV_dask(ridge, param_grid=param_grid, n_jobs=-1, cv = 10, scoring="neg_mean_squared_error")
grid_ml.fit(X_ml, y_ml)

CPU times: user 254 ms, sys: 73.7 ms, total: 328 ms
Wall time: 3min 12s


In [17]:
print("Loss: %f Mejor modelo: %s"%(grid_ml.best_score_, grid_ml.best_params_))

Loss: -0.039179 Mejor modelo: {'C': 0.01}


Tambien se implementa utilizando la funcion Ridge usada anteriormente y el CV de dask

In [20]:
%%time
param_grid = {
    "alpha": [0.01, 0.03, 0.05, 0.07, 0.09, 0.1]
}
ridge_2 = Ridge(max_iter=100)
grid_ml_2 = GridSearchCV_dask(ridge_2, param_grid=param_grid, n_jobs=-1, cv = 10, scoring="neg_mean_squared_error")
grid_ml_2.fit(X_ml, y_ml)

CPU times: user 31.2 ms, sys: 0 ns, total: 31.2 ms
Wall time: 456 ms


In [22]:
print("Loss: %f Mejor modelo: %s"%(grid_ml_2.best_score_, grid_ml_2.best_params_))

Loss: -0.016413 Mejor modelo: {'alpha': 0.03}


### Conclusiones

De los resultados anteriores puedo concluir que combinar el estimador de sklearn con el GridSearchCV de dask-ml  fue la opcion mas rapida (no esta muy claro qué hace esta combinacion o si es conveniente hacerla pero se intento por el mal desempeño de dask-ml). El segundo mas rapido fue hacer el grid en paralelo utilizando "delayed". La tercera opcion fue utilizando sklearn secuencial y en un ultimo lugar utilizando dask-ml. Se comprobo ademas que en todos los casos (menos con dask-ml) se alcanzo la misma loss. Es notable ademas la gran diferencia entre dask-ml y los demas ya que fue casi 60 veces mas lento.

Es importante recalcar que existe una gran varianza en los tiempos de ejecucion. En algunas ocaciones (pocas) el orden establecido anteriormente fue diferente.

### Bonus

Se implemento ademas el mismo problema en Spark (ver otro notebook). Spark fue considerablemente mas lento que el secuencial, en paralelo con "delayed" y la combinacion de sklearn y dask-ml.